In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import torch.utils.data
import torch.nn
from random import randrange
import os
os.environ["WDS_VERBOSE_CACHE"] = "1"

In [2]:
import webdataset as wds

In [4]:
url = "/share/projset/laion5B-data/laion2b_multi_data/img_data_filter/{000000..000010}.tar"
dataset = wds.WebDataset(url).shuffle(1000).decode("rgb").to_tuple("png", "json")

In [5]:
for image, json in dataset:
    break
plt.imshow(image)

ValueError: didn't find ['png'] in ['__key__', '__url__', '0.png', '0.txt']

In [16]:
import ast
import glob
import json
import logging
import math
import os
import random
import sys
import time
from dataclasses import dataclass
import io
from multiprocessing import Value
from transformers import XLMRobertaTokenizer
import braceexpand
import numpy as np
import pandas as pd
import torch
import torchvision.datasets as datasets
import webdataset as wds
from PIL import Image, TarIO
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler, IterableDataset, get_worker_info
from torch.utils.data.distributed import DistributedSampler
from webdataset.filters import shuffle
from webdataset.tariterators import base_plus_ext, url_opener, tar_file_expander, valid_sample
from transformers import AutoTokenizer, AltCLIPProcessor

import json
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True # Truncated File Read
Image.MAX_IMAGE_PIXELS = None # DecompressionBombWarning

M18_LANGUAGES = ['ar', 'de', 'en', 'es', 'fr', 'hi', 'it', 'ja', 'ko', 'nl', 'pl', 'pt', 'ru', 'th', 'tr', 'uk', 'vi', 'zh']
M9_LANGUAGES = ['ar', 'en', 'es', 'fr', 'it', 'ja', 'ko',  'ru', 'zh']

try:
    import horovod.torch as hvd
except ImportError:
    hvd = None

from open_clip import tokenize

class SharedEpoch:
    def __init__(self, epoch: int = 0):
        self.shared_epoch = Value('i', epoch)

    def set_value(self, epoch):
        self.shared_epoch.value = epoch

    def get_value(self):
        return self.shared_epoch.value


@dataclass
class DataInfo:
    dataloader: DataLoader
    sampler: DistributedSampler = None
    shared_epoch: SharedEpoch = None

    def set_epoch(self, epoch):
        if self.shared_epoch is not None:
            self.shared_epoch.set_value(epoch)
        if self.sampler is not None and isinstance(self.sampler, DistributedSampler):
            self.sampler.set_epoch(epoch)


def preprocess_txt(text):
    return text


def get_dataset_size(shards):
    shards_list = list(braceexpand.braceexpand(shards))
    dir_path = os.path.dirname(shards)
    sizes_filename = os.path.join(dir_path, 'sizes.json')
    len_filename = os.path.join(dir_path, '__len__')
    if os.path.exists(sizes_filename):
        sizes = json.load(open(sizes_filename, 'r'))
        total_size = sum([int(sizes[os.path.basename(shard)]) for shard in shards_list])
    elif os.path.exists(len_filename):
        # FIXME this used to be eval(open(...)) but that seemed rather unsafe
        total_size = ast.literal_eval(open(len_filename, 'r').read())
    else:
        total_size = None  # num samples undefined
        # some common dataset sizes (at time of authors last download)
        # CC3M (train): 2905954
        # CC12M: 10968539
        # LAION-400M: 407332084
        # LAION-2B (english): 2170337258
    num_shards = len(shards_list)
    return total_size, num_shards

def count_samples(dataloader):
    os.environ["WDS_EPOCH"] = "0"
    n_elements, n_batches = 0, 0
    for images, texts in dataloader:
        n_batches += 1
        n_elements += len(images)
        assert len(images) == len(texts)
    return n_elements, n_batches

def decode_webdataset(src):
    for sample in src:
        print(sample)
        try:
            if "0.png" in sample:
                sample["png"] = sample["0.png"]
                del sample["0.png"]
            if "0.txt" in sample:
                sample["txt"] = sample["0.txt"].decode("utf-8")
                del sample["0.txt"]
            img = sample["png"]
            with io.BytesIO(img) as stream:
                img = Image.open(stream).convert("RGB")
            sample["png"] = img    
            yield sample
        except Exception as e:
            # print(str(e))
            continue

def filter_no_caption(sample):
    return 'txt' in sample

def filter_no_caption_or_no_image_or_notm18(sample):
    has_caption = ('txt' in sample or '0.txt' in sample)
    has_image = ('png' in sample or 'jpg' in sample or 'jpeg' in sample or 'webp' in sample or '0.png' in sample)
    is_18_language = False
    if has_caption:
        decode_str = sample["txt"].decode("utf-8") if 'txt' in sample else sample['0.txt'].decode("utf-8")
        caption, language = decode_str.split('\t') 
        is_18_language = language in M9_LANGUAGES
    return has_caption and has_image and is_18_language

def filter_no_caption_or_no_image(sample):
    has_caption = ('txt' in sample or '0.txt' in sample)
    has_image = ('png' in sample or 'jpg' in sample or 'jpeg' in sample or 'webp' in sample or '0.png' in sample)
    return has_caption and has_image 

def log_and_continue(exn):
    """Call in an exception handler to ignore any exception, isssue a warning, and continue."""
    logging.warning(f'Handling webdataset error ({repr(exn)}). Ignoring.')
    return True


def group_by_keys_nothrow(data, keys=base_plus_ext, lcase=True, suffixes=None, handler=None):
    """Return function over iterator that groups key, value pairs into samples.

    :param keys: function that splits the key into key and extension (base_plus_ext)
    :param lcase: convert suffixes to lower case (Default value = True)
    """
    current_sample = None
    for filesample in data:
        assert isinstance(filesample, dict)
        fname, value = filesample["fname"], filesample["data"]
        prefix, suffix = keys(fname)
        if prefix is None:
            continue
        if lcase:
            suffix = suffix.lower()
        # FIXME webdataset version throws if suffix in current_sample, but we have a potential for
        #  this happening in the current LAION400m dataset if a tar ends with same prefix as the next
        #  begins, rare, but can happen since prefix aren't unique across tar files in that dataset
        if current_sample is None or prefix != current_sample["__key__"] or suffix in current_sample:
            if valid_sample(current_sample):
                yield current_sample
            current_sample = dict(__key__=prefix, __url__=filesample["__url__"])
        if suffixes is None or suffix in suffixes:
            current_sample[suffix] = value
    if valid_sample(current_sample):
        yield current_sample


def tarfile_to_samples_nothrow(src, handler=log_and_continue):
    # NOTE this is a re-impl of the webdataset impl with group_by_keys that doesn't throw
    streams = url_opener(src, handler=handler)
    files = tar_file_expander(streams, handler=handler)
    samples = group_by_keys_nothrow(files, handler=handler)
    return samples


def pytorch_worker_seed():
    """get dataloader worker seed from pytorch"""
    worker_info = get_worker_info()
    if worker_info is not None:
        # favour the seed already created for pytorch dataloader workers if it exists
        return worker_info.seed
    # fallback to wds rank based seed
    return wds.utils.pytorch_worker_seed()


_SHARDshLE_SIZE = 2000
_SHARDshLE_INITIAL = 500
_SAMPLEshLE_SIZE = 5000
_SAMPLEshLE_INITIAL = 1000


class detshuffle2(wds.PipelineStage):
    def __init__(
            self,
            bufsize=1000,
            initial=100,
            seed=0,
            epoch=-1,
    ):
        self.bufsize = bufsize
        self.initial = initial
        self.seed = seed
        self.epoch = epoch

    def run(self, src):
        if isinstance(self.epoch, SharedEpoch):
            epoch = self.epoch.get_value()
        else:
            # NOTE: this is epoch tracking is problematic in a multiprocess (dataloader workers or train)
            # situation as different workers may wrap at different times (or not at all).
            self.epoch += 1
            epoch = self.epoch
        rng = random.Random()
        if self.seed < 0:
            seed = pytorch_worker_seed() + epoch
        else:
            seed = self.seed + epoch
        rng.seed(seed)
        return shle(src, self.bufsize, self.initial, rng)


class ResampledShards2(IterableDataset):
    """An iterable dataset yielding a list of urls."""

    def __init__(
        self,
        urls,
        nshards=sys.maxsize,
        worker_seed=None,
        deterministic=False,
        epoch=-1,
    ):
        """Sample shards from the shard list with replacement.

        :param urls: a list of URLs as a Python list or brace notation string
        """
        super().__init__()
        urls = wds.shardlists.expand_urls(urls)
        self.urls = urls
        assert isinstance(self.urls[0], str)
        self.nshards = nshards
        self.rng = random.Random()
        self.worker_seed = pytorch_worker_seed if worker_seed is None else worker_seed
        self.deterministic = deterministic
        self.epoch = epoch

    def __iter__(self):
        """Return an iterator over the shards."""
        if isinstance(self.epoch, SharedEpoch):
            epoch = self.epoch.get_value()
        else:
            # NOTE: this is epoch tracking is problematic in a multiprocess (dataloader workers or train)
            # situation as different workers may wrap at different times (or not at all).
            self.epoch += 1
            epoch = self.epoch
        if self.deterministic:
            # reset seed w/ epoch if deterministic, worker seed should be deterministic due to arg.seed
            self.rng.seed(self.worker_seed() + epoch)
        for _ in range(self.nshards):
            yield dict(url=self.rng.choice(self.urls))


def get_wds_dataset(args, preprocess_img, is_train, epoch=0, floor=False):
    input_shards = args.train_data if is_train else args.val_data
    assert input_shards is not None
    resampled = getattr(args, 'dataset_resampled', False) and is_train
    
    num_samples, num_shards = get_dataset_size(input_shards)
    if not num_samples:
        if is_train:
            num_samples = args.train_num_samples
            if not num_samples:
                raise RuntimeError(
                    'Currently, number of dataset samples must be specified for training dataset. '
                    'Please specify via `--train-num-samples` if no dataset length info present.')
        else:
            num_samples = args.val_num_samples or 0  # eval will just exhaust the iterator if not specified

    shared_epoch = SharedEpoch(epoch=epoch)  # create a shared epoch store to sync epoch to dataloader worker proc
    if resampled:
        pipeline = [ResampledShards2(input_shards, deterministic=True, epoch=shared_epoch)]
    else:
        pipeline = [wds.SimpleShardList(input_shards)]

    # at this point we have an iterator over all the shards
    if is_train:
        if not resampled:
            pipeline.extend([
                detshuffle2(
                    bufsize=_SHARDshLE_SIZE,
                    initial=_SHARDshLE_INITIAL,
                    seed=args.seed,
                    epoch=shared_epoch,
                ),
                wds.split_by_node,
                wds.split_by_worker,
            ])
        pipeline.extend([
            # at this point, we have an iterator over the shards assigned to each worker at each node
            tarfile_to_samples_nothrow,  # wds.tarfile_to_samples(handler=log_and_continue),
            wds.shuffle(
                bufsize=_SAMPLEshLE_SIZE,
                initial=_SAMPLEshLE_INITIAL,
            ),
        ])
    else:
        pipeline.extend([
            wds.split_by_worker,
            # at this point, we have an iterator over the shards assigned to each worker
            wds.tarfile_to_samples(handler=log_and_continue),
        ])
    # tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large")
    processor = AltCLIPProcessor.from_pretrained("/share/project/chenzhongzhi/multi-clip/multi-clip/ckpt/kd_m18_1and2_fromscratch/checkpoint-258420")
    tokenizer = processor.tokenizer
    if 'laion2b' in args.train_data:
        pipeline.extend([
            wds.select(filter_no_caption_or_no_image_or_notm18),
            decode_webdataset,
            wds.rename(image="jpg;png;jpeg;webp;0.png", text="txt;0.txt"),
            wds.map_dict(image=preprocess_img, text=lambda text: tokenizer(text,padding='max_length',max_length=77,truncation=True,return_tensors='pt')),
            wds.to_tuple("image", "text"),
            wds.batched(args.batch_size, partial=not is_train),
        ]) 
    else:
        pipeline.extend([
            wds.select(filter_no_caption_or_no_image),
            wds.decode("pilrgb", handler=log_and_continue),
            wds.rename(image="jpg;png;jpeg;webp", text="txt"),
            wds.map_dict(image=preprocess_img, text=lambda text: tokenizer(text,padding='max_length',max_length=77,truncation=True,return_tensors='pt')),
            wds.to_tuple("image", "text"),
            wds.batched(args.batch_size, partial=not is_train),
        ])
    # pipeline.extend([
    #     wds.select(filter_no_caption),
    #     wds.decode("pilrgb", handler=log_and_continue),
    #     wds.rename(image="jpg;png", text="txt"),
    #     wds.map_dict(image=preprocess_img, text=preprocess_txt),
    #     wds.to_tuple("image", "text"),
    #     wds.batched(args.batch_size, partial=not is_train),
    # ])

    dataset = wds.DataPipeline(*pipeline)
    if is_train:
        if not resampled:
            assert num_shards >= args.workers * args.world_size, 'number of shards must be >= total workers'
        # roll over and repeat a few samples to get same number of full batches on each node
        round_fn = math.floor if floor else math.ceil
        global_batch_size = args.batch_size * args.world_size
        num_batches = round_fn(num_samples / global_batch_size)
        num_workers = max(1, args.workers)
        num_worker_batches = round_fn(num_batches / num_workers)  # per dataloader worker
        num_batches = num_worker_batches * num_workers
        num_samples = num_batches * global_batch_size
        dataset = dataset.with_epoch(num_worker_batches)  # each worker is iterating over this
    else:
        # last batches are partial, eval is done on single (master) node
        num_batches = math.ceil(num_samples / args.batch_size)

    dataloader = wds.WebLoader(
        dataset,
        batch_size=None,
        shuffle=False,
        num_workers=args.workers,
        persistent_workers=True,
    )

    # FIXME not clear which approach is better, with_epoch before vs after dataloader?
    # hoping to resolve via https://github.com/webdataset/webdataset/issues/169
    # if is_train:
    #     # roll over and repeat a few samples to get same number of full batches on each node
    #     global_batch_size = args.batch_size * args.world_size
    #     num_batches = math.ceil(num_samples / global_batch_size)
    #     num_workers = max(1, args.workers)
    #     num_batches = math.ceil(num_batches / num_workers) * num_workers
    #     num_samples = num_batches * global_batch_size
    #     dataloader = dataloader.with_epoch(num_batches)
    # else:
    #     # last batches are partial, eval is done on single (master) node
    #     num_batches = math.ceil(num_samples / args.batch_size)

    # add meta-data to dataloader instance for convenience
    dataloader.num_batches = num_batches
    dataloader.num_samples = num_samples

    return DataInfo(dataloader=dataloader, shared_epoch=shared_epoch)


def get_dataset_fn(data_path, dataset_type):
    if dataset_type == "webdataset":
        return get_wds_dataset
    
def get_data(args, preprocess_fns, epoch=0):
    preprocess_train, preprocess_val = preprocess_fns
    data = {}

    if args.train_data:
        data["train"] = get_dataset_fn(args.train_data, args.dataset_type)(
            args, preprocess_train, is_train=True, epoch=epoch)

    if args.val_data:
        data["val"] = get_dataset_fn(args.val_data, args.dataset_type)(
            args, preprocess_val, is_train=False)

    return data


In [18]:
def world_info_from_env():
    local_rank = 0
    for v in ('LOCAL_RANK', 'MPI_LOCALRANKID', 'SLURM_LOCALID', 'OMPI_COMM_WORLD_LOCAL_RANK'):
        if v in os.environ:
            local_rank = int(os.environ[v])
            break
    global_rank = 0
    for v in ('RANK', 'PMI_RANK', 'SLURM_PROCID', 'OMPI_COMM_WORLD_RANK'):
        if v in os.environ:
            global_rank = int(os.environ[v])
            break
    world_size = 1
    for v in ('WORLD_SIZE', 'PMI_SIZE', 'SLURM_NTASKS', 'OMPI_COMM_WORLD_SIZE'):
        if v in os.environ:
            world_size = int(os.environ[v])
            break

    return local_rank, global_rank, world_size

class OPT():
    def __init__(self) -> None:
        pass
    
arg = OPT()
arg.train_data = "/share/projset/laion5B-data/laion2b_multi_data/img_data_filter/{000000..000010}.tar"
arg.dataset_type = "webdataset"
arg.val_data = None
arg.train_num_samples = 2000
arg.seed = 3407
arg.batch_size = 4
arg.workers = 6
arg.dataset_resampled = True
arg.local_rank, arg.rank, arg.world_size = world_info_from_env()

In [3]:
import torchvision.transforms as transforms

normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                std=[0.5, 0.5, 0.5])

tf = transforms.Compose( [
        transforms.Resize((512,512)),
        transforms.ToTensor(),
        normalize
    ] ) 

In [20]:
data = get_data(arg, (tf, tf), epoch=0)

True


In [27]:
loader = data['train'].dataloader

In [32]:
batch = next(iter(loader))
batch[0].size()

torch.Size([32, 3, 512, 512])

In [33]:
isinstance(loader, IterableDataset)

True

In [34]:
shards = "/share/projset/laion5B-data/laion2b_multi_data/img_data_filter/{000000..000010}.tar"
list(braceexpand.braceexpand(shards))

['/share/projset/laion5B-data/laion2b_multi_data/img_data_filter/000000.tar',
 '/share/projset/laion5B-data/laion2b_multi_data/img_data_filter/000001.tar',
 '/share/projset/laion5B-data/laion2b_multi_data/img_data_filter/000002.tar',
 '/share/projset/laion5B-data/laion2b_multi_data/img_data_filter/000003.tar',
 '/share/projset/laion5B-data/laion2b_multi_data/img_data_filter/000004.tar',
 '/share/projset/laion5B-data/laion2b_multi_data/img_data_filter/000005.tar',
 '/share/projset/laion5B-data/laion2b_multi_data/img_data_filter/000006.tar',
 '/share/projset/laion5B-data/laion2b_multi_data/img_data_filter/000007.tar',
 '/share/projset/laion5B-data/laion2b_multi_data/img_data_filter/000008.tar',
 '/share/projset/laion5B-data/laion2b_multi_data/img_data_filter/000009.tar',
 '/share/projset/laion5B-data/laion2b_multi_data/img_data_filter/000010.tar']

In [36]:
dir_path = os.path.dirname(shards)
dir_path

'/share/projset/laion5B-data/laion2b_multi_data/img_data_filter'

In [1]:
from multiprocessing import Value
shared_epoch = Value('i', 0)

In [2]:
shared_epoch

<Synchronized wrapper for c_int(0)>

In [5]:
def test():
    urls = ['a', 'b', 'c']

    for url in urls:
        yield dict(url=url)

In [13]:
for i in range(3):
    print(next(iter(test())))

{'url': 'a'}
{'url': 'a'}
{'url': 'a'}


In [ ]:
 wds.ResampledShards(url)